In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
import sqlite3

In [3]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key
def insMain(d):
    dbname='mknews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from mknews where nkey="'+d[6]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into mknews (ncate,ntitle,ncomp,ndate,nurl,ntext,nkey) values(?,?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')

In [4]:
headers={'authority': "search.naver.com",
             "method": "GET",
             "path": '/search.naver?&where=news&query=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=1&start=11&refresh_start=0',
             "scheme": "https",
             "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
             "accept-encoding": "gzip, deflate, br",
             "accept-language": "ko,en-US;q=0.9,en;q=0.8",
             "cache-control": "max-age=0",
             "cookie": "NNB=AEP7EKDQOVCWA; nx_ssl=2; nid_inf=740873661; NID_AUT=hRvGg8ABo6CAUZLNPau2fqKxSOI2Jm6Sk+xfNTpcuMrE8CUUKSPnRkCEbBEJ+eVV; NID_JKL=LQqDnEnSrpuFc9mJHiiqLGNZShX8/8HeQ5sb63lKCb4=; _ga=GA1.2.1521198919.1615261089; ASID=3a4c8213000001782476ee550000005d; NDARK=Y; nso_open=1; news_my_status=1; news_office_checked=1009; NID_SES=AAABtTIZqPJJhx7ulMXz7TSiGa65nV5WjcMWlBPTcle4SCqq5iyyQv5OXhWgNeoZzNxr2krcrw4X8ppKBCmJCM6TUV/RlEVDDDQzJpVzDeQytD1qzNEwn/qI1T2uDc/TotacqB9L7qQfr3gE19wsZH2GKIkvMkQrgNfer8VXc0hQrWyBfzaoFuCrivQMl55JlC/Ink5gXsHjZGyrVSPgc4hRMD849sgUS+WpmULuRtWiwKYTHP1CzDYWBs65DZvu3xrDsX4PfJGC0jX73c1fBHQjAG/waO1U64WsrEHielRPB7LKVa3FhXiTcQ+J7u4gviO9rZG7sUUlC5zkhC+GdtuTLegG3bfC/MYiBBXRABpqj0MLCF2+GNQ8TV3xj9mxLSkXqCF5nRvTvp7tbDSf1WdzZpM/fvPBbwdwCBU5V4uFNS9KB1NjLdxLGBXeYX4e/VpmUMxmvGqN3QvG3amP7FNRGA5byX1KqqMk4YuqOAF8CkBjjZVS8ww/rz3KsZdYT9mHUU5yckBDgdCGm+63b7YGxAOQJJ15oPQpBD0yhnj8mZeGtTV6ruXDlNnRvOr99VsTN3S+p4A7uFKiCGCFI5SJA8A=; page_uid=haKcDsp0JywssNl/e8GssssssRo-223963; _naver_usersession_=vieuFR5fdoU74Bj3g41WIw==; BMR=s=1616054353349&r=https%3A%2F%2Fm.blog.naver.com%2FPostView.nhn%3FblogId%3Dkiddwannabe%26logNo%3D221185808375%26proxyReferer%3Dhttps%3A%252F%252Fwww.google.com%252F&r2=https%3A%2F%2Fwww.google.com%2F",
             "sec-ch-ua-mobile": "?0",
             "sec-fetch-dest": "document",
             "sec-fetch-mode": "navigate",
             "sec-fetch-site": "none",
             "sec-fetch-user": "?1",
             "upgrade-insecure-requests": "1",
             "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"}
c=0
for y in range(1,4001,10):
    url='https://search.naver.com/search.naver?&where=news&query=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=1&start={}&refresh_start=0'.format(y)
    res=requests.get(url,headers=headers)
    src=res.content.strip()
    html =bsp(src,'html.parser')
    tgts=html.find(attrs={'class':'list_news'})
    list_tgts=tgts.find_all('li')
    print('10개씩')
    for i in list_tgts:
        try:
            #매일경제
            url=i.find(attrs={'class':'dsc_thumb '})['href']
            res=requests.get(url,headers={'User-Agent':'Mozilla/5.0'})
            src=res.content.strip()
            key=makeKey(url)
            html =bsp(src,'html.parser')
            title1=html.find(attrs={'class':'top_title'}).text
            title2=html.find(attrs={'class':'sub_title1_new'}).text
            #author=html.find(attrs={'class':'author'}).text.strip().replace('\xa0',' ')
            time=html.find(attrs={'class':'lasttime'}).text
            time=re.sub('수정.*','',time).replace('입력 :','').replace('\xa0','').strip()
            content=html.find(attrs={'class':'art_txt'}).text
            content=content.replace('\n',"").replace('\t',"").replace('\r',"")
            content=re.sub('window.*','',content)
            content=re.sub('googletag.*;','',content)
            content=re.sub('\[.*?\]','',content)
            content=re.sub('◆.*?◆','',content)
            content=content.replace('▶ 여기를 누르시면 크게 보실 수 있습니다','')
            content=content.strip()
            title=title1+title2
            print('#########')
            comp='매일경제'
            opt=1
            print(c)
            c+=1
            #d=(opt,title,comp,time,url,content,key)
            #insMain(d)
        except:
            pass

10개씩
#########
0
#########
1
#########
2
#########
3
#########
4
#########
5
#########
6
#########
7
10개씩
#########
8
#########
9
#########
10
#########
11
#########
12
#########
13
#########
14
#########
15
10개씩
#########
16
#########
17
#########
18
#########
19
#########
20
#########
21
#########
22
10개씩
#########
23
#########
24
#########
25
#########
26
#########
27
#########
28
#########
29
10개씩
#########
30
#########
31
#########
32
#########
33
#########
34
#########
35
#########
36
10개씩
#########
37
#########
38
#########
39
#########
40
#########
41
#########
42


KeyboardInterrupt: 

In [ ]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key
def insMain(d):
    dbname='mknews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from mknews where nkey="'+d[6]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into mknews (ncate,ntitle,ncomp,ndate,nurl,ntext,nkey) values(?,?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
key=makeKey(url)
comp='매일경제'
opt=1
d=(opt,title,comp,time,url,content,key)
insMain(d)

In [7]:
#매일경제
url='http://news.mk.co.kr/newsRead.php?no=138050&year=2020'
res=requests.get(url,headers={'User-Agent':'Mozilla/5.0'})
src=res.content.strip()
key=makeKey(url)
html =bsp(src,'html.parser')
title1=html.find(attrs={'class':'top_title'}).text
title2=html.find(attrs={'class':'sub_title1_new'}).text
#author=html.find(attrs={'class':'author'}).text.strip().replace('\xa0',' ')
time=html.find(attrs={'class':'lasttime'}).text
time=re.sub('수정.*','',time).replace('입력 :','').replace('\xa0','').strip()
content=html.find(attrs={'class':'art_txt'}).text
content=content.replace('\n',"").replace('\t',"").replace('\r',"")
content=re.sub('window.*','',content)
content=re.sub('googletag.*;','',content)
content=re.sub('\[.*?\]','',content)
content=re.sub('◆.*?◆','',content)
content=content.replace('▶ 여기를 누르시면 크게 보실 수 있습니다','')
#content=re.sub('[^가-힣a-z0-9]','',content)◆
content=content.strip()
title=title1+title2
comp='매일경제'
opt=1
#d=(opt,title,comp,time,url,content,key)
#insMain(d)
print(content)
#print(' 제목 : '+title1+title2+' 기자 : '+author+' 게시시간 : '+time+' 본문 : '+content)

"섣불리 시작했다가 공장 내 근로자 한 명만 확진돼도 보름 이상 공장을 통째로 스톱시켜야 합니다. 엄청난 리스크지만 중국 근로자도 출근하고 있고 더 이상 공장을 놀릴 수는 없잖아요."(화학 업체 관계자) 세계의 공장인 중국이 신종 코로나바이러스 감염증(우한 폐렴) 사태 속에 10일 조업 재개를 실시하면서 삼성·현대차·SK·LG 등 중국에서 생산공장을 가동 중인 주요 기업들은 비상경영 상태로 조업 준비와 가동에 들어갔다. 조속한 재개가 시급하지만 여전히 확진자와 사망자가 빠르게 증가하고 있어 공장 가동에 상당히 조심스러운 분위기다. 중국에서 석유화학과 배터리를 비롯해 9개 공장을 운영하고 있는 LG화학은 이날부터 출근 가능한 직원들이 조업에 복귀했다. 24시간 동안 가동되는 석유화학 장치 산업 특성상 LG화학 석유화학 공장들은 춘제 기간 가동률을 줄여서 운영해왔다. 중국 자싱시·허페이시·싼장 지역에서 석유화학 공장을 가동 중인 롯데케미칼도 연휴가 끝나는 10일에 맞춰 주재원들이 복귀했으며 이날 출근이 가능한 인력부터 일터로 복귀했다. 회사 관계자는 "감염으로 인해 행여나 공장 내에서 확진자가 발생하지 않도록 한국 본사와 수시로 연락하며 비상경영을 하고 있다"고 전했다. 중국 정부의 요청에 따라 공장 재가동 및 정상화를 위한 서류 작업이 진행 중인 곳도 있어 석유화학 업계에선 공장 정상 가동은 이달 말이나 돼야 할 것으로 예상하고 있다. 한 기업 관계자는 "공장 내에서 확진자가 발생하면 모든 공장을 셧다운하고 방역해야 하는데 이럴 경우 피해가 만만치 않을 것으로 예상되는 만큼 신경을 곤두세우고 있다"며 "이번주 중국 내 상황이 진전되기를 바라고 있다"고 말했다. 애플 아이폰을 조립 생산하는 대만 폭스콘은 허난성 정저우 공장 재가동을 예정일인 10일에서 일주일 더 늦춰 17일 이후 본격 가동에 들어갈 것으로 보인다. 폭스콘은 그간 근무 중 반드시 마스크를 착용하고 매일 수차례 체온 검사를 실시했다. 심지어 마스크 부족 사태로 인해 직원들이 쓸 마스크를 직접 생산하는

In [ ]:
#경향신문
url='http://biz.khan.co.kr/khan_art_view.html?artid=202103162149005&code=920501'
res=requests.get(url,headers={'User-Agent':'Mozilla/5.0'})
src=res.content.strip()
html =bsp(src,'html.parser')
title=html.find(attrs={'class':'headline'}).text
author=html.find(attrs={'class':'name'}).text
author=re.sub('[^가-힣]','',author)
time=html.find(attrs={'class':'byline'}).text.replace('\n','')
time=re.sub('수정.*','',time).replace('입력 :','').replace('\xa0','').strip()
content=html.find(attrs={'class':'art_body'}).text
author=re.sub('[^가-힣]','',author)
content=content.replace('\n',"").replace('\t',"").replace('\r',"")
content=re.sub('createIframe.*;','',content)
print(' 제목 : '+title+' 기자 : '+author+' 게시시간 : '+time+' 본문 : '+content)

In [ ]:
#경향신문
url='http://www.hani.co.kr/arti/economy/consumer/987007.html'
res=requests.get(url,headers={'User-Agent':'Mozilla/5.0'})
src=res.content.strip()
html =bsp(src,'html.parser')
title=html.find_all(attrs={'class':'title'})#.text.replace('\n',"").replace('\t',"").replace('\r',"")
title
# author=html.find(attrs={'align':'right'}).text
# author=re.sub('[^가-힣]','',author)
#time=html.find(attrs={'align':'right'}).text.replace('\n','')
#time
# time=re.sub('수정.*','',time).replace('입력 :','').replace('\xa0','').strip()
# content=html.find(attrs={'class':'art_body'}).text
# author=re.sub('[^가-힣]','',author)
# content=content.replace('\n',"").replace('\t',"").replace('\r',"")
# content=re.sub('createIframe.*;','',content)
# print(' 제목 : '+title+' 기자 : '+author+' 게시시간 : '+time+' 본문 : '+content)

In [ ]:
url='https://www.mk.co.kr/news/business/view/2021/03/256458/'
res=requests.get(url,headers={'User-Agent':'Mozilla/5.0'})
src=res.content.strip()
html =bsp(src,'html.parser')
title1=html.find(attrs={'class':'top_title'})
title2=html.find(attrs={'class':'sub_title1_new'})
author=html.find(attrs={'class':'author'})
time=html.find(attrs={'class':'lasttime'})
content=html.find(attrs={'class':'art_txt'}).text
content=content.replace('\n',"").replace('\t',"").replace('\r',"")
t=re.sub('\[.*?\]',' ',content)
#t=re.sub('[googletag.*;]',' ',t)
print(t)